In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
train_data = pd.read_csv('train.csv')
# 前两列是名称不是数据, 后24列为一天24小时的测量值
# 每一行代表某天某测量指标一天的测量值
test_data = pd.read_csv('test.csv')
test = test_data[test_data['AMB_TEMP'] == 'PM2.5']
# 待预测的测试集x
test_x = test.iloc[:, 2:]
train = train_data[train_data['columns'] == 'PM2.5'].iloc[:, 2:]
train_x = []
train_y = []
# 将每一天的PM2.5的数据以10为一组划分,前9个为x, 第10个为y,例如0..9是x,10为y.
# 最后一组是14..22是x,23为y
for i in range(15):
    x = train.iloc[:, i:i+9].values
    y = train.iloc[:, i+9].values
    train_x.append(x)
    train_y.append(y)

# 测试数据
train_x = np.vstack(train_x)  # 3600 x 9
train_y = np.hstack(train_y)  # 3600
train_x = np.array(train_x,float)
train_y = np.array(train_y,float)

In [12]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
lr = RidgeCV(alphas=alphas, fit_intercept=True, normalize=True, cv=10)
lr.fit(train_x, train_y)
r2 = r2_score(lr.predict(train_x), train_y)
print(r2)
print(lr.alpha_)

0.8302040150831448
0.0001


In [18]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    train_x, train_y, test_size=0.2, shuffle=True, random_state=123)
degrees = [2, 3, 4, 5]
scores = []
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
for degree in degrees:
    model = make_pipeline(PolynomialFeatures(degree),
                          RidgeCV(alphas=alphas, fit_intercept=False, normalize=True, cv=10))
    model.fit(X_train, y_train)
    scores.append(r2_score(model.predict(X_test), y_test))
print(scores)

D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.23684e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.18992e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.22477e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.17008e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.17728e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAl

D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.17519e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.13275e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.15053e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.2044e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.38282e-18): result may not be accurate.
  overwrite_a=True).T
D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlg

[0.836409397157984, 0.7365127751040221, -0.01097969801409393, -0.004014536110284128]


In [19]:
lr1 = make_pipeline(PolynomialFeatures(2),RidgeCV(alphas=alphas, fit_intercept=False,normalize=True,cv=10))
lr1.fit(train_x,train_y)
r2_score(lr1.predict(train_x),train_y)

0.8431088283415815

In [20]:
sampleSubmission = pd.read_csv('sampleSubmission.csv',index_col='id')
sampleSubmission['value'] = lr1.predict(test_x)
sampleSubmission.to_csv('result.csv')